In [ ]:

import pandas as pd


In [ ]:


from pymongo import MongoClient

url_servidor = 'mongodb://127.0.0.1:27017/'


client = MongoClient(url_servidor)

# código para ver si se ha conectado bien
try:
    s = client.server_info() # si hay error tendremos una excepción
    print("Conectado a MongoDB, versión",s["version"])
    db = client["PD1"]
except:
    print ("Error de conexión ¿está arrancado el servidor?")

In [ ]:
df = pd.read_csv("MTA_Subway_Stations.csv")
df


In [ ]:

columnas_utiles = ['GTFS Stop ID', 'Stop Name', 'Daytime Routes', 'GTFS Longitude', 'GTFS Latitude']
df_limpio = df[columnas_utiles].copy()


df_limpio = df_limpio.rename(columns={
    'GTFS Stop ID': 'GTFS_id_estacion',
    'Stop Name': 'nombre',
    'Daytime Routes': 'lineas'
})


df_limpio['ubicacion'] = df_limpio.apply(
    lambda fila: {
        "type": "Point",
        "coordinates": [fila['GTFS Longitude'], fila['GTFS Latitude']]
    }, axis=1
)


df_limpio = df_limpio.drop(columns=['GTFS Longitude', 'GTFS Latitude'])


documentos_para_mongo = df_limpio.to_dict(orient='records')

In [ ]:
df_limpio

In [ ]:
db.subway.drop()

In [ ]:
db.subway.insert_many(documentos_para_mongo)


In [ ]:
from rich import print_json
from json import dumps

# muestra el contenido de un cursor
def muestra(cursor):
    # Convertir a JSON y mostrar con rich
    i = 0
    for doc in cursor:
        i+=1
        print_json(dumps(doc, indent=4, default=str))
    return i

In [29]:
muestra(db.subway.find({}).limit(1))

{
  "_id": "699499623bd4787667e92d2a",
  "GTFS_id_estacion": "127",
  "nombre": "Times Sq-42 St",
  "lineas": "1 2 3",
  "ubicacion": {
    "type": "Point",
    "coordinates": [
      -73.987495,
      40.75529
    ]
  }
}

1

In [23]:
db.subway.drop_indexes()

In [ ]:
db.subway.create_index({"ubicacion":"2dsphere"})

'ubicacion_2dsphere'

In [ ]:

def cursor_paradas_afectedas(coordinates): #coordinates de esta forma [longitud, latitud]
    cursor = db.subway.find(
   {
     "ubicacion":
       { "$near" :
          {
            "$geometry": { "type": "Point",  "coordinates": coordinates },
            "$maxDistance": 500
          }
       }
   }
)
    return cursor


#muestra(cursor)

In [35]:
def extraccion_paradas(cursor):
    afectadas = []
    for doc in cursor:
        afectadas.append((doc["nombre"], doc["lineas"]))

    return afectadas


In [ ]:
#ejemplo
afectadas = extraccion_paradas(cursor_paradas_afectedas([-73.9866, 40.7592]))
afectadas

[('49 St', 'N R W'),
 ('42 St-Port Authority Bus Terminal', 'A C E'),
 ('Times Sq-42 St', 'S'),
 ('50 St', '1'),
 ('50 St', 'C E'),
 ('Times Sq-42 St', '7'),
 ('Times Sq-42 St', '1 2 3'),
 ('47-50 Sts-Rockefeller Ctr', 'B D F M')]